In [64]:
import codecs
f=codecs.open("NLP_TWEET_1.html", 'r')
print (f.read())


<!doctype html>
<html lang="en">
<head>
    <meta content="text/html;charset=UTF-8" http-equiv="Content-Type" />
    <script src='https://s3.amazonaws.com/mturk-public/externalHIT_v1.js' type='text/javascript'></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.5.0/css/bootstrap.min.css" integrity="sha256-aAr2Zpq8MZ+YA/D6JtRD3xtrwpEz2IqOS+pWD/7XKIw=" crossorigin="anonymous" />
    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.5.1/jquery.min.js" integrity="sha256-9/aliU8dGd2tb6OSsuzixeV4y/faTqgFtohetphbbj0=" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/PapaParse/5.1.0/papaparse.min.js" integrity="sha256-Fh801SO9gqegfUdkDxyzXzIUPWzO/Vatqj8uN+5xcL4=" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.5.0/js/bootstrap.min.js" integrity="sha256-OFRAJNoaD8L3Br5lglV7VyLRf0itmoBzWUoM+Sji4/8=" crossorigin="anonymous"></script>


In [13]:
print(f.readline()[0])

ValueError: I/O operation on closed file.

In [65]:
texts=[]
with open("NLP_TWEET_1.html", "r", encoding='utf-8') as f:
    text= f.readlines()
    print(len(text))
    #texts = text.split('\n')
    print(text[2])
    texts.append(text)
texts = texts[0]

82
<html lang="en">



In [70]:
texts[58]

'          let randomNum = getRandomInt(results.data.length);\n'

In [71]:
for i in range(377):
    texts[58] = '          let randomNum = '+str(i)+';\n'
    with open(str(i)+'.html', 'w') as outfile:
        for lines in texts:
            outfile.write(lines)

In [11]:
from __future__ import division, unicode_literals 
import codecs
from bs4 import BeautifulSoup

f=codecs.open("NLP_TWEET_1.html", 'r', 'utf-8')
document= BeautifulSoup(f.read()).get_text()
print (document)

ModuleNotFoundError: No module named 'bs4'

In [36]:
import pandas as pd
import numpy as np
train_file = pd.read_csv("https://gist.githubusercontent.com/bshmueli/c99fc0abf56460e644bd610bf3024e83/raw/720285d133c85d94e0aa3fe3edcc199f6d99e3f7/lab4-data.csv")

In [47]:
train_file.columns

Index(['idx', 'text'], dtype='object')

In [53]:
temp=0
for i in range(len(train_file)):
    temp +=len(train_file.iloc[i,1])
temp = temp/len(train_file)
reward = [1.0 if len(i)>=temp else 0.5  for i in train_file['text']]

In [51]:
import boto3
import pandas as pd

CREDENTIALS_FILE = 'credentials.csv'
credentials = pd.read_csv(CREDENTIALS_FILE).to_dict('records')[0]
aws_access_key_id = credentials['Access key ID']
aws_secret_access_key = credentials['Secret access key']

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

# Uncomment this line to use in production
# endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
 
client = boto3.client(
    'mturk',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url=endpoint_url,
    region_name=region_name
)

In [52]:
print(client.get_account_balance())

{'AvailableBalance': '10000.00', 'ResponseMetadata': {'RequestId': '168c7631-868f-4f60-be34-877ad7aa79d1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '168c7631-868f-4f60-be34-877ad7aa79d1', 'content-type': 'application/x-amz-json-1.1', 'content-length': '31', 'date': 'Mon, 25 May 2020 06:36:39 GMT'}, 'RetryAttempts': 0}}


In [60]:
one_minute = 60 # seconds
one_hour = 60 * one_minute
one_day = 24 * one_hour
#qualification.private
hit_type_id = []
for i in range(10):
    hit_type_response = client.create_hit_type(
          AutoApprovalDelayInSeconds=10 * one_minute,
          AssignmentDurationInSeconds=5 * one_minute,
          Reward=str(reward[i]),
          Title='NCTU NLP TEST',
          Keywords='color,language,fun',
          Description='Choose the color that most matches its name',
          QualificationRequirements=[
                  {
                  'QualificationTypeId': '00000000000000000060',
                  'Comparator': 'EqualTo',
                  'IntegerValues': [1],
              },
              {
                  'QualificationTypeId': '00000000000000000071', # see https://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
                  'Comparator': 'In',
                  'LocaleValues': [
                      {
                          'Country': 'TW'
                      },
                  ],
                  'RequiredToPreview': True,
                  'ActionsGuarded': 'PreviewAndAccept'
              },
          ]
      )
    
    hit_type_id.append(hit_type_response['HITTypeId'])

In [62]:
question='''<?xml version="1.0" encoding="UTF-8"?>
<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
  <ExternalURL>https://omelette-314.github.io/Deep_learning_nctu/NLP online/'''+str(1)+'''.html</ExternalURL>
  <FrameHeight>800</FrameHeight>
</ExternalQuestion>'''

response = client.create_hit_with_hit_type(
    HITTypeId=hit_type_id[0],
    MaxAssignments=3,
    LifetimeInSeconds=1 * one_hour,
    Question=question,
    RequesterAnnotation='NLP',
)

In [63]:
print(question)

<?xml version="1.0" encoding="UTF-8"?>
<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
  <ExternalURL>https://omelette-314.github.io/Deep_learning_nctu/NLP online/1.html</ExternalURL>
  <FrameHeight>800</FrameHeight>
</ExternalQuestion>
